In [1]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil



import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline
import logging
from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
#os.environ['TPU_LOAD_LIBRARY']='0'
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
import tensorflow as tf


import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
from scipy.stats.stats import spearmanr  
## custom modules
import metrics as metrics
from optimizers import *
import schedulers as schedulers

import training_utils

import enformer_performer as enformer_performer

from scipy import stats


2023-02-22 02:54:33.119396: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 02:54:33.286953: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-22 02:54:33.286986: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-22 02:54:34.192734: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='node-6')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy=\
        tf.data.experimental.AutoShardPolicy.FILE
    options.deterministic=False
    options.experimental_threading.max_intra_op_parallelism=1
    mixed_precision.set_global_policy('mixed_bfloat16')
    #options.num_devices = 64

    BATCH_SIZE_PER_REPLICA = 1 # batch size 24, use LR ~ 2.5 e -04
    NUM_REPLICAS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_REPLICAS
    

2023-02-22 02:54:35.622373: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-22 02:54:35.622426: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-22 02:54:35.622455: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-genformer-v2-6): /proc/driver/nvidia/version does not exist


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


2023-02-22 02:54:35.916226: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 02:54:35.940766: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:447] Started server with target: grpc://localhost:45622
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: node-6


INFO:tensorflow:Initializing the TPU system: node-6


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
g = tf.random.Generator.from_non_deterministic_state()
with strategy.scope():
    iterators={'human': ('gs://genformer_data/expanded_originals/196k',5313),
               'mouse': ('gs://genformer_data/expanded_originals/196k',1643)}


    tr_data_it_dict,val_data_it_dict,val_data_TSS_it =  \
        training_utils.return_distributed_iterators(iterators,
                                                    "gs://genformer_data/expanded_originals/196k/human/tfrecords_tss",
                                                     GLOBAL_BATCH_SIZE,
                                                     196608,
                                                     10,
                                                     4,
                                                     10,
                                                     strategy,
                                                     options,
                                                     g)


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [4]:
with strategy.scope():
    model = enformer_performer.enformer_performer(
        num_transformer_layers=8,
        num_heads=8,
        heads_channels= {'human': 5313,
                         'mouse': 1643},
        dim=258,
        d_model=2064,
        norm=False,
        max_seq_length=1536,
        nb_random_features=256,
        filter_list_seq=[192,192,192,192,192,2048],
        hidden_size=2064,
        numerical_stabilizer=0.001,
        rel_pos_bins=1536,
        use_mask_pos=False,
        use_rot_emb=True,
        stable_variant=True,
        freeze_conv_layers=False,
        kernel_transformation="relu_kernel_transformation",
        conv_block_type='enformer',
        normalize=True,
        seed=5)

In [5]:
with strategy.scope():        

    optimizer1 = tf.keras.optimizers.Adam(learning_rate=1.0e-04)

    optimizer2 = tf.keras.optimizers.Adam(learning_rate=1.0e-04)
    optimizers_in = optimizer1,optimizer2

    metric_dict = {}
    organism_dict = {'human': (1000,100),
                     'mouse': (1000,100)}

    dist_train_step, dist_val_step_h,dist_val_step_m, val_step_TSS, build_step,metric_dict = training_utils.return_train_val_functions(model,
                                                                                                                                         150,
                                                                                                                                       50,
                                                                                                                                       50,
                                                                                                                                         500,
                                                                                                                                         optimizers_in,
                                                                                                                                         strategy,
                                                                                                                                         metric_dict, 
                                                                                                                                         GLOBAL_BATCH_SIZE,
                                                                                                                                        0.20,
                                                                                                                                         BATCH_SIZE_PER_REPLICA,
                                                                                                                                        loss_fn_main='poisson')

    
    
    

In [ ]:
with strategy.scope():        
    ### main training loop
    global_step = 0
    val_losses = []
    val_pearsons = []
    val_R2 = []
    patience_counter = 0
    stop_criteria = False
    best_epoch = 0

    for epoch_i in range(1, 4):
        print('starting epoch_', str(epoch_i))
        start = time.time()
        if epoch_i == 1:
            # run once to build the model w/o updating anything
            build_step(val_data_it_dict['human'])

        total_params=0
        for k in model.trainable_variables:
            var = k.values[0]
            total_params += tf.size(var)
        print('total params: ' + str(total_params)) 
        
        dist_train_step(tr_data_it_dict['human'],
                        tr_data_it_dict['mouse'])
        


        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i))
        print('hg_train_loss: ' + str(metric_dict['human_tr'].result().numpy()))
        
        print('training duration(mins): ' + str(duration))

        start = time.time()
        dist_val_step_h(val_data_it_dict['human'])

        print('val_loss: ' + str(metric_dict['human_val'].result().numpy()))
        val_losses.append(metric_dict['human_val'].result().numpy())

        print('human_pearsonsR: ')
        pearsonsR=np.nanmean(metric_dict['human_pearsonsR'].result()['PearsonR'].numpy())
        print(pearsonsR)

        val_pearsons.append(np.nanmedian(pearsonsR))
        print('human_R2: ')
        print(np.nanmean(metric_dict['human_R2'].result()['R2'].numpy()))


        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i) + ' validation')
        print('validation duration(mins): ' + str(duration))
        print('patience counter at: ' + str(patience_counter))

        
        val_step_TSS(val_data_TSS_it)
        

        y_trues = metric_dict['hg_corr_stats'].result()['y_trues'].numpy()
        y_preds = metric_dict['hg_corr_stats'].result()['y_preds'].numpy()
        cell_types = metric_dict['hg_corr_stats'].result()['cell_types'].numpy()
        gene_map = metric_dict['hg_corr_stats'].result()['gene_map'].numpy()
        
        
        
        if (epoch_i > 2):
            stop_criteria,patience_counter,best_epoch = \
                training_utils.early_stopping(current_val_loss=val_losses[-1],
                                                logged_val_losses=val_losses,
                                                current_pearsons=val_pearsons[-1],
                                                logged_pearsons=val_pearsons,
                                                current_epoch=epoch_i,
                                                best_epoch=best_epoch,
                                                save_freq=5,
                                                patience=5,
                                                patience_counter=patience_counter,
                                                min_delta=1.0e-05,
                                                model=enformer_model,
                                                save_directory="gs://picard-testing-176520/test",
                                                saved_model_basename="test_model",
                                                checkpoint=checkpoint)
        #plt.close('all')
        print('patience counter at: ' + str(patience_counter))
        for key, item in metric_dict.items():
            item.reset_state()

starting epoch_ 1


/opt/conda/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer RandomUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


total params: tf.Tensor(321435484, shape=(), dtype=int32)


In [ ]:
[x.name for x in model.layers]

In [20]:
total_params=0
for k in model.stem_conv_atac.trainable_variables:
    var = k.values[0]
    total_params += tf.size(var)
print('total params: ' + str(total_params)) 

total params: tf.Tensor(2272, shape=(), dtype=int32)


In [27]:
model.layers[3].trainable_variables

[TPUDistributedVariable:{
   0: <tf.Variable 'conv_downsample_1/layer_normalization_3/gamma:0' shape=(768,) dtype=float32, numpy=
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1.

In [24]:
model.conv_tower.layers

AttributeError: 'conv_next_tower' object has no attribute 'layers'

PerReplica:{
  0: <tf.Tensor: shape=(1, 896, 5313), dtype=float16, numpy=
array([[[0.01155 , 0.02716 , 0.0206  , ..., 0.      , 0.      ,
         0.      ],
        [0.0109  , 0.04666 , 0.0309  , ..., 0.      , 0.      ,
         0.      ],
        [0.00792 , 0.035   , 0.01932 , ..., 0.      , 0.      ,
         0.      ],
        ...,
        [0.      , 0.006863, 0.      , ..., 0.      , 0.      ,
         0.      ],
        [0.      , 0.      , 0.002575, ..., 0.      , 0.0795  ,
         0.      ],
        [0.014786, 0.      , 0.03992 , ..., 0.      , 0.1423  ,
         0.      ]]], dtype=float16)>,
  1: <tf.Tensor: shape=(1, 896, 5313), dtype=float16, numpy=
array([[[0.09924 , 0.0927  , 0.01834 , ..., 0.      , 0.      ,
         0.      ],
        [0.11127 , 0.1686  , 0.03397 , ..., 0.      , 0.9844  ,
         0.      ],
        [0.1432  , 0.2322  , 0.01851 , ..., 0.      , 0.      ,
         0.      ],
        ...,
        [0.00663 , 0.01672 , 0.007565, ..., 0.01852 , 0.11566 ,


In [2]:
def exponential_linspace_int(start, end, num, divisible_by=1):
  """Exponentially increasing values of integers."""
  def _round(x):
    return int(np.round(x / divisible_by) * divisible_by)

  base = np.exp(np.log(end / start) / (num - 1))
  return [_round(start * base**i) for i in range(num)]

In [3]:
print(exponential_linspace_int(192,384,6))

[192, 221, 253, 291, 334, 384]
